In [1]:
import argparse

import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.model_selection import KFold

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import GModel
from myutils import roc_auc, translate_result
from optimizer import Optimizer
from sampler import BalancedSampler

In [3]:
class Args:
    def __init__(self):
        self.device = "cpu"  # cuda:number or cpu
        self.data = "gdsc1"  # Dataset{gdsc or ccle}

args = Args()
device = args.device

In [4]:
res, drug_feature, exprs, mut, cna, null_mask = load_data(args)

load gdsc1
response matrix (res) shape: (300, 925)
exprs shape: (1084, 19562)
mut shape: (1084, 18099)
cna shape: (1084, 24502)
925
exprs shape: (925, 19562)
mut shape: (925, 18099)
cna shape: (925, 24502)
drug_feature shape: (300, 920)
response matrix (res) shape: (925, 300)
null_mask shape: (925, 300)


In [5]:
res

,(-)-Parthenolide,(5Z)-7-Oxozeaenol,5-Fluorouracil,681640,A-443654,A-484954,A-770041,A-83-01,ACY-1215,AGI-6780,...,Y-39983,YK 4-279,YM201636,Z-LLNle-CHO,Z-Leu-leu-leu-al,ZG-10,ZM-447439,ZSTK474,Zibotentan,kb NB 142-70
201T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
22RV1,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
23132-87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42-MG-BA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0
451Lu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YT,NaN,NaN,1.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.0,NaN,NaN
ZR-75-30,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
huH-1,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
no-10,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [6]:
adj_coo = sp.coo_matrix(res)
pos_edges = np.vstack((adj_coo.row, adj_coo.col)).T

# ネガティブエッジ候補作成
adj_dense = adj_coo.toarray()

# ポジティブ（接続ありかつ null_mask==0）
pos_mask = np.logical_and(adj_dense == 1, null_mask == 0)
pos_row, pos_col = np.where(pos_mask)
pos_edges = np.vstack((pos_row, pos_col)).T
pos_labels = np.ones(len(pos_edges), dtype=int)

# ネガティブ（接続なしかつ null_mask==0）
neg_mask = np.logical_and(adj_dense == 0, null_mask == 0)
neg_row, neg_col = np.where(neg_mask)
neg_edges = np.vstack((neg_row, neg_col)).T
neg_labels = np.zeros(len(neg_edges), dtype=int)

# 統合
all_edges = np.vstack((pos_edges, neg_edges))
all_labels = np.concatenate((pos_labels, neg_labels))

In [7]:
exprs.shape

(925, 19562)

In [8]:
len(all_labels)

31084

In [9]:
k = 5
kf = KFold(n_splits=5, shuffle=True, random_state=42)

true_datas = pd.DataFrame()
predict_datas = pd.DataFrame()

for i, (train_idx, test_idx) in enumerate(kf.split(all_edges)):
    train_edges = all_edges[train_idx]
    train_labels = all_labels[train_idx]
    test_edges = all_edges[test_idx]
    test_labels = all_labels[test_idx]

    sampler = BalancedSampler(
        edge_train=train_edges,
        label_train=train_labels,
        edge_test=test_edges,
        label_test=test_labels,
        adj_shape=res.shape,
    )
    
    model = GModel(
        adj_mat=sampler.train_data,
        gene=exprs,
        cna=cna,
        mutation=mut,
        sigma=2,
        k=11,
        iterates=3,
        feature_drug=drug_feature,
        n_hid1=192,
        n_hid2=36,
        alpha=5.74,
        device=device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        roc_auc,
        lr=5e-4,
        epochs=1000,
        device=device,
    ).to(device)
    epoch, true_data, predict_data = opt()
    true_datas = pd.concat([true_datas, translate_result(true_data)])
    predict_datas = pd.concat([predict_datas, translate_result(predict_data)])

epoch:   0 loss:0.729831 auc:0.4814
epoch:  20 loss:0.650317 auc:0.7396
epoch:  40 loss:0.611453 auc:0.7590
epoch:  60 loss:0.536569 auc:0.7952
epoch:  80 loss:0.513621 auc:0.8082
epoch: 100 loss:0.499076 auc:0.8152
epoch: 120 loss:0.496930 auc:0.8168
epoch: 140 loss:0.488388 auc:0.8202
epoch: 160 loss:0.480309 auc:0.8230
epoch: 180 loss:0.483112 auc:0.8216
epoch: 200 loss:0.450760 auc:0.8308
epoch: 220 loss:0.431279 auc:0.8355
epoch: 240 loss:0.397249 auc:0.8436
epoch: 260 loss:0.367679 auc:0.8529
epoch: 280 loss:0.318555 auc:0.8604
epoch: 300 loss:0.285404 auc:0.8682
epoch: 320 loss:0.260432 auc:0.8747
epoch: 340 loss:0.256694 auc:0.8787
epoch: 360 loss:0.241667 auc:0.8794
epoch: 380 loss:0.227742 auc:0.8847
epoch: 400 loss:0.218957 auc:0.8865
epoch: 420 loss:0.212876 auc:0.8882
epoch: 440 loss:0.207962 auc:0.8895
epoch: 460 loss:0.204935 auc:0.8909
epoch: 480 loss:0.201080 auc:0.8918
epoch: 500 loss:0.197937 auc:0.8933
epoch: 520 loss:0.198887 auc:0.8938
epoch: 540 loss:0.193868 auc

In [10]:
true_datas.to_csv(f"true_{args.data}.csv")
predict_datas.to_csv(f"pred_{args.data}.csv")